In [1]:
#mount gdrive. notice: the dataset should be uploaded already
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import math
import pandas as pd

In [3]:
questions=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/questions_w_tag.csv', lineterminator='\n')
behaviors=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/behaviors_w_tag.csv', lineterminator='\n')

In [4]:
#fit the tfidf model
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b").fit_transform(questions['content'])

In [5]:
#calculate the similarity
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_model[3], tfidf_model).flatten()
cosine_similarities

array([0.09797634, 0.        , 0.0859316 , ..., 0.06468089, 0.04901487,
       0.1116779 ])

使用餘弦相似度來找到與某篇文章最相近的文章代號

In [137]:
#given a doc_index, find top k similar documents
def find_sim_doc(index,rank):
  flag=0
  place=0
  cosine_similarities = linear_kernel(tfidf_model[index], tfidf_model).flatten()
  arr=np.array(cosine_similarities)
  sim=arr.argsort()[rank*-1:][::-1]
  for i in range(len(sim)):
    if sim[i]==index:
      place=i
      flag+=1
  if flag==1:
    return(np.delete(sim,place))
  else:
    return(sim)
    
find_sim_doc(4000,2)[-1]

2295

檢視該方法成效

In [7]:
#see how well the recommndation performs
index=1900
print('viewed: \n'+ questions['content'][index])
print('推薦為：')
sim=find_sim_doc(index, 5)
for i in range(len(sim)):
  print(questions['content'][sim[i]])


viewed: 
各 位 學長姊 好 我 目前 在 一 家 大 外商 公司 做 銷售 兼 客服 的 助理 已經 年 的 時間 因為 最近 畢業 想 找 正職 工作 我 本身 非常 喜歡 面對 人 、 幫助 人 、 接受 挑戰 等等 我 覺得 自己 滿 適合 業務 的 工作 內容 之前 有 一 個 機會 在 台灣 企業 工作 非常 不能 接受 工作 的 氣氛 、 制度 （ 我 了解 一 間 公司 不能 代表 全部 或許 有些 外商 也 是 一樣 但 目前 的 經驗 還是 覺得 有 差別 的 ） 所以 這 間 公司 我 之前 有 投 過 業務 的 職缺 （ 福利 跟 薪水 真的 滿 好 的 而且 我 真的 非常 有 興趣 ） 雖然 差點 面試 成功 後來 因為 沒有 實質 的 銷售 經驗 公司 選擇 了 另 一 位 有 經驗 的 現在 公司 又 開 客服 職缺 相較 之下 沒有 很 有 興趣 不 想 把 客服 作為 整 個 工作 內容 的 全部 （ 平常 上班 看 客服 人員 的 工作 非常 的 爆量 跟 痛苦 ） 也 害怕 未來 想 轉職 業物 性質 的 工作 是 完全 沒有 幫助 的 （ 目前 自己 不會 想 長時間 做 客服 ） 但 正 如 剛剛 所 說 的 薪水 跟 福利 都 非常 吸引人 有 聽 別人 說 第一 份 工作 的 薪水 很 重要 會 決定 未來 談 薪水 的 價格 還 有 因為 上面 提到 的 偏向 找 外商 有 聽說 外商 需要 國外 學歷 本身 學歷 是 私立 科大 想 說 是否 該 藉 此 機會 洗 一下 經歷 來 彌補 學歷 非 外國 真的 讓 我 非常 猶豫 ⋯⋯⋯ 拜託 拜託 請 幫 我 分析 該 如何 面對 🙏🏻
推薦為：
您 好 我 目前 歲 是 應屆 畢業生 的 新鮮人 過去 的 工作 經驗 不 算 多 也 沒有 長期 的 實習 經驗 唯一 一 次 的 實習 是 五專 時期 的 世大運 實習 因為 自己 選擇 的 學制 的 關係 自己 的 主修 也 處於 曖昧 不明 的 狀態 我 是 五專 應用 外文系 畢業 後來 又 念 了 兩 年 的 二技 設計系 目前 找 的 工作 類型 都 是 社群 行銷 、 行銷 企劃 類型 的 工作 是 自己 有 興趣 的 工作 雖然 沒有 很多 相關 的 實質 工作 經驗 但 過去 的 幾 年 皆 

該使用者有 銷售 客服 經驗， 想找業務性質工作 的求職請益 

1. 推薦1:社群媒體 經驗 求職請益
2. 推薦2:服務業 餐飲業 業務經驗 想找 作業員 的求職請益
3. 推薦3:企管系 客服 經驗 找 轉職 的求職請益
4. 推薦4:文案 遊戲測試 保全 文字客服 找 文案/行銷/企劃/客服 的求職請益

以上推薦皆有類似經歷/目標，因此判定該推薦方法可行

接著計算使用者在每類別文章中的閱讀偏好統計量

In [64]:
def get_distribution(pid):
  tmp=[0,0,0,0,0]
  specific_behaviors=behaviors.loc[behaviors['pid']==pid]
  all_doc=specific_behaviors['question_id'].unique()


  for j in range(len(all_doc)):
    #avoid the same question
    tag=specific_behaviors.loc[behaviors['question_id']==all_doc[j]]['tag'].unique()[0]
    tmp[tag]+=1
  return tmp

get_distribution('2384cc6e-2389-4be1-bc0d-84d682ea2714')


[0, 11, 5, 1, 5]

輸出為[590, 479, 1274, 862, 1069]，可見該使用者在類2中的互動頻率最高，類4次之，以此類推

接著根據以上分佈，來分配總額度，由於需求中為至少10篇，因此以10進行測試

In [72]:
#according to the user's behavior to assign the recommend quota for each tag
def assign_quota(pid,all_quota):
  quota=[]
  distribution=get_distribution(pid)
  for i in range(len(distribution)):
    quota.append(distribution[i]/sum(distribution)*all_quota)
  
  #because high proportion of documents are tagged as cluster 1, so i assign at least one quota to different cluster
  #this might due to a failed-trained Knn model, or the dataset itself is imbalance
  for j in range(len(quota)):
    quota[j]=math.ceil(quota[j])

  return quota
assign_quota('2384cc6e-2389-4be1-bc0d-84d682ea2714',10)

[0, 5, 3, 1, 3]

最後輸出為[0,5,3,1,3]，即類0推零篇，類1推五篇，以此類推 （推0篇因為該user從未看過該類文章）

計算使用者所互動過的文章中，各文章的分數，該分數計分方法可自行定義，在此處由於閱讀、按讚是較為容易的動作，因此設置為0.2分，thank為0.5，share 跟attention則更高，為0.8（需要進行使用者行為的分析、研究，在此值之設定上才可以更為準確）

In [81]:
def calculate_point(pid):
  #all the behaviors of the specific user
  specific_behaviors=behaviors.loc[behaviors['pid']==pid]
  #distinct dociuments of this user viewed/clapped...etc.
  unique_doc=specific_behaviors['question_id'].unique()
  q_dict={}
  q_score=[]

  for q in unique_doc:
    score=0
    q_dict[q]=score
    specific_behaviors_q=specific_behaviors.loc[specific_behaviors['question_id']==q]
    q_tag=specific_behaviors_q['tag'][specific_behaviors_q.index].values
    specific_behaviors_q=specific_behaviors_q['event'][specific_behaviors_q.index]

    for i in range(len(specific_behaviors_q.values)):
      event=specific_behaviors_q.values[i]
      if event=='view':
        score+=0.2
      if event=='clap':
        score+=0.2
      if event=='attention':
        score+=0.8
      if event=='share':
        score+=0.8
      if event=='thank':
        score+=0.5
      q_dict[q]+=score
  return(q_dict)
(calculate_point('2384cc6e-2389-4be1-bc0d-84d682ea2714'))


{'02f883e2-09fb-4a46-afa8-bca4f4dc50e1': 0.2,
 '1b3da501-10d5-469c-9362-461ea27050d3': 2.0,
 '1d5133cd-0293-4148-8382-80b9532708df': 0.6000000000000001,
 '2758c3ac-7d9a-45f7-b75f-f9c2fa52dae5': 0.2,
 '2b247eb3-07e5-4112-9764-ad0afafe49b5': 0.2,
 '2dcb7829-437c-41e9-b7df-f3abf5e323a3': 0.2,
 '430ab193-9a37-43c6-8220-13047604dad8': 3.0,
 '4707846a-563a-49fe-a8a9-3d4fa62fd0dc': 0.6000000000000001,
 '5a358548-5574-4cc2-8b89-16caac2ee10e': 0.2,
 '5d652049-b4ab-4f0a-b358-330e2c5acb75': 2.0,
 '6eb6fcc4-2d2c-4b21-ab55-d9e3d91d8613': 1.2000000000000002,
 '6fa11ad1-aff7-48b3-bd0b-b385d858ed38': 1.2000000000000002,
 '75d6026b-d4f4-4fd3-9bbd-08b13c0916e5': 0.6000000000000001,
 '7708685d-bf0d-4b38-b26c-eecabbd92ddd': 0.6000000000000001,
 'b0b5b2db-ad12-4c08-b4a8-16e0cd3567dc': 0.2,
 'b8208b05-b0dc-11e8-b307-065035b8ff22': 0.6000000000000001,
 'c3c59ddc-1b88-4241-a0d3-f6161ea0bd0d': 0.2,
 'cd386473-56da-41e4-bd8e-9fef5b056d2a': 0.2,
 'e4ad34c8-c87d-4f37-b057-f961b8776c63': 0.2,
 'e67ba9d4-bcb9-4934-

最後之輸出為該使用者閱讀過的文章中，其文章之分數

接著將各問題分到其對應的cluster

In [152]:
def arrange_question(pid):
  
  q_dict={0:[],1:[],2:[],3:[],4:[]}
  point=calculate_point(pid)
  q_list=list(calculate_point(pid))
  sort_q_list=[]

  #sort the questions with its point
  sort_orders = sorted(point.items(), key=lambda x: x[1], reverse=True)
  for i in range(len(sort_orders)):
    sort_q_list.append(list(sort_orders[i])[0])

  for i in range(len(sort_q_list)):
    tag=((behaviors.loc[behaviors['question_id']==sort_q_list[i]])['tag']).unique()[0]
    if tag in q_dict:
      q_dict[tag].append(sort_q_list[i])
    if tag not in q_dict:
      q_dict[tag]=[sort_q_list[i]]
  return q_dict

arrange_question('2384cc6e-2389-4be1-bc0d-84d682ea2714')


{0: [],
 1: ['430ab193-9a37-43c6-8220-13047604dad8',
  'e8c1344d-da8b-4f73-b3c0-5ff3ca7a28da',
  '5d652049-b4ab-4f0a-b358-330e2c5acb75',
  '1b3da501-10d5-469c-9362-461ea27050d3',
  '4707846a-563a-49fe-a8a9-3d4fa62fd0dc',
  'b8208b05-b0dc-11e8-b307-065035b8ff22',
  '75d6026b-d4f4-4fd3-9bbd-08b13c0916e5',
  'c3c59ddc-1b88-4241-a0d3-f6161ea0bd0d',
  '5a358548-5574-4cc2-8b89-16caac2ee10e',
  'e4ad34c8-c87d-4f37-b057-f961b8776c63',
  '2dcb7829-437c-41e9-b7df-f3abf5e323a3'],
 2: ['6eb6fcc4-2d2c-4b21-ab55-d9e3d91d8613',
  '6fa11ad1-aff7-48b3-bd0b-b385d858ed38',
  '1d5133cd-0293-4148-8382-80b9532708df',
  '2b247eb3-07e5-4112-9764-ad0afafe49b5',
  '02f883e2-09fb-4a46-afa8-bca4f4dc50e1'],
 3: ['cd386473-56da-41e4-bd8e-9fef5b056d2a'],
 4: ['7708685d-bf0d-4b38-b26c-eecabbd92ddd',
  'e67ba9d4-bcb9-4934-bc3a-e9592a28f23e',
  'f9e6be74-e142-491f-b607-0974d9d264fd',
  'b0b5b2db-ad12-4c08-b4a8-16e0cd3567dc',
  '2758c3ac-7d9a-45f7-b75f-f9c2fa52dae5']}

最後則是整合以上資訊，進行推薦

In [132]:
def recommend(pid):
  arranged_question=arrange_question(pid)
  all_recommends_index=[]
  all_recommends=[]
  quota=assign_quota(pid,30)
  type_priority=np.argsort(quota)[::-1]
  type_quantity=np.sort(quota)[::-1]
  point=calculate_point(pid)
  q_list=list(point)
  #sort the questions with its point
  sort_orders = sorted(point.items(), key=lambda x: x[1], reverse=True)
  
  for doc in range(len(q_list)):
    tag=((behaviors.loc[behaviors['question_id']==q_list[doc]])['tag']).unique()[0]
    rec=find_sim_doc(questions.loc[questions['question_id']==q_list[doc]].index,2)
    if quota[tag]!=0:
      quota[tag]-=1
      all_recommends_index.append(rec)
    if quota[tag]==0 and len(all_recommends_index)<10:
      all_recommends_index.append(rec)  
  for k in range(len(all_recommends_index)):
    title=questions['title'][k]
    all_recommends.append(title)
  return(all_recommends)
recommend('2384cc6e-2389-4be1-bc0d-84d682ea2714')

  



('430ab193-9a37-43c6-8220-13047604dad8', 3.0)


[nan,
 'R&D Engineer',
 '請益職場相處技巧，很少主動開口與同事聊天是否會造成同事反感',
 nan,
 '目前還是學生，想找工程類型工作，應徵都無下文',
 '測試發問測試發問測試發問測試發問測試發問測試發問測試發問',
 '育嬰留停期間又生產可再留停嗎?',
 nan,
 '之前工作被資遣，面試新工作被問離職原因時該怎麼回答比較好？',
 '當設計SOHO族比較好，還是去公司當朝九晚五的上班族比較好？',
 '被資遣面試新工作被問離職原因回答方式',
 '失業兩周沒自信繼續求職  沒有目標該怎麼辦',
 '兩家公司面試該如何抉擇',
 '我想轉職，但沒有個方向，想請問各位的想法',
 '職場其實是不想錄用中年大叔',
 '請問準備國考的這件事要寫在自傳裡嗎?',
 '想轉換跑道，嘗試其他工作',
 '該傾聽自己出去闖天下，還是傾聽家人放棄夢想?',
 '想嘗試其他工作，轉換跑道',
 '不想做作業員，但離目標工作還好遠',
 '想轉職至建築設計/室內設計，想知道必備條件是那些?',
 '稽核轉資安的入門及後續規劃']

最後輸出對該使用者的推薦文章

存在的問題：


1.   使用者若歷史互動文章未滿10篇，則推薦文章數會<10
2.   在閱讀比例的部分，對於閱讀比例高的類別，其優先度並沒有更高，排序方法仍以[0,1,2,3,4]類排列

解決方案：


1.   若推薦未滿10篇，則根據該使用者閱讀過的文章，再推薦第二相似的文章(find_dim_doc方法中的rank參數），直到推薦滿10篇為止 （ＥＸ：該使用者只閱讀過一篇文章，在推薦最相近文章後仍未滿10篇推薦，則再推薦第二相似的文章，此時仍未滿10篇，則重複迭代以上流程，直到滿10篇為止）
2.   對於quota先進行排序再進行推薦，如對user在0~4類的推薦quota為[1,3,3,2,1]，經排序後為[3,3,1,2,1]，則先推薦第1、2類，再推薦第3類，接著才是0和4類




================================================================================================================================================================================================================================================================================================================================

改進：


*   新增arrange_question 將每個問題分別歸類到其cluster之下
*   新增變數type_priority來排序type的優先順序 （解決方案2)
*   當使用者閱讀數量不足 仍能推薦到指定的數量 （解決方案1)




In [ ]:
def recommend_beta(pid,numbers):
  arranged_question=arrange_question(pid)
  all_recommends_index=[]
  all_recommends=[]
  quota=assign_quota(pid,numbers)
  type_priority=np.argsort(quota)[::-1]
  type_quantity=np.sort(quota)[::-1]
  point=calculate_point(pid)
  q_list=list(point)
  #sort the questions with its point
  sort_orders = sorted(point.items(), key=lambda x: x[1], reverse=True)

  for i in range(len(type_priority)):
    current_type=type_priority[i]
    quantity=type_quantity[i]
    current_state=0
    current_rank=2
    questions_of_type=arranged_question[current_type]
    if quantity==0:
      break
    while quantity!=0:
      if len(questions_of_type)-1==current_state:
        rec=find_sim_doc(questions.loc[questions['question_id']==questions_of_type[current_state]].index,current_rank)[-1]
        all_recommends_index.append(rec)
        quantity-=1
        current_rank+=1
        current_state=0
      else:
        rec=find_sim_doc(questions.loc[questions['question_id']==questions_of_type[current_state]].index,current_rank)[-1]
        all_recommends_index.append(rec)
        quantity-=1
        current_state+=1 
  for k in range(len(all_recommends_index)):
    title=questions['title'][k]
    all_recommends.append(title)
  return(all_recommends)     
      
        

回覆主管問題：
問題1：需有輸入與輸出（在初版即有此功能 call recommend(pid)) 輸入 pid 輸出推薦文章

在更新版中則為 recommend_beta(pid) 輸入pid 輸出推薦文章


問題2:推薦數量的更動 在初版已可透過assign_quota來指定要推薦的篇數（但可能在推薦時會推薦不足）

該問題已於beta中解決

問題3:bug 修復

已針對提出之解決方案進行更新 詳見recommend_beta

以下為該function demo



In [158]:
#針對該user推薦80篇
recommend_beta('2384cc6e-2389-4be1-bc0d-84d682ea2714',80)

[nan,
 'R&D Engineer',
 '請益職場相處技巧，很少主動開口與同事聊天是否會造成同事反感',
 nan,
 '目前還是學生，想找工程類型工作，應徵都無下文',
 '測試發問測試發問測試發問測試發問測試發問測試發問測試發問',
 '育嬰留停期間又生產可再留停嗎?',
 nan,
 '之前工作被資遣，面試新工作被問離職原因時該怎麼回答比較好？',
 '當設計SOHO族比較好，還是去公司當朝九晚五的上班族比較好？',
 '被資遣面試新工作被問離職原因回答方式',
 '失業兩周沒自信繼續求職  沒有目標該怎麼辦',
 '兩家公司面試該如何抉擇',
 '我想轉職，但沒有個方向，想請問各位的想法',
 '職場其實是不想錄用中年大叔',
 '請問準備國考的這件事要寫在自傳裡嗎?',
 '想轉換跑道，嘗試其他工作',
 '該傾聽自己出去闖天下，還是傾聽家人放棄夢想?',
 '想嘗試其他工作，轉換跑道',
 '不想做作業員，但離目標工作還好遠',
 '想轉職至建築設計/室內設計，想知道必備條件是那些?',
 '稽核轉資安的入門及後續規劃',
 '面試設計相關職缺，履歷和作品集要怎麼準備？如何提升面試機會?',
 '轉型成前端設計師會不會比較有"錢"途？',
 nan,
 '請問倘若有前置協商的情況會影響求職的機會嗎？',
 '想了解作業流程與職責歸屬問題，以及該如何修飾離職原因',
 '想轉換跑道，嘗試其他工作',
 '面試幾間公司的業助，幾乎都是無聲卡，下步不知該怎麼走',
 '想轉職科技業卻不知能從何開始',
 '31歲沒有工作方向，投履歷都無下文',
 '受到上市公司邀約，是否應該跳槽?',
 '想轉換跑道，嘗試其他工作',
 nan,
 '實施分流工作，是否算職務調動',
 '針對疫情，對應屆畢業生的就業建議和該注意什麼\n',
 '一個待業中的年輕人，求職方向指點。',
 '在想是否想投入推拿按摩產業',
 '想轉換跑道，嘗試其他工作',
 nan,
 '待業中，想詢問是否該繼續找相同職缺但放低標準，還是轉換跑道',
 '如何降低經營時的威脅性？',
 '想轉換跑道，嘗試其他工作',
 '畢業三年，經歷兩份採購相關工作，想要轉職。',
 '想詢問出社會轉職，跳到另一個領域，該做什麼準備',
 '如何成為人上人，成為Design Leader的必備條件？'

In [159]:
#只會推薦多於80篇
len(recommend_beta('2384cc6e-2389-4be1-bc0d-84d682ea2714',80))

82

In [160]:
recommend_beta('2384cc6e-2389-4be1-bc0d-84d682ea2714',30)

[nan,
 'R&D Engineer',
 '請益職場相處技巧，很少主動開口與同事聊天是否會造成同事反感',
 nan,
 '目前還是學生，想找工程類型工作，應徵都無下文',
 '測試發問測試發問測試發問測試發問測試發問測試發問測試發問',
 '育嬰留停期間又生產可再留停嗎?',
 nan,
 '之前工作被資遣，面試新工作被問離職原因時該怎麼回答比較好？',
 '當設計SOHO族比較好，還是去公司當朝九晚五的上班族比較好？',
 '被資遣面試新工作被問離職原因回答方式',
 '失業兩周沒自信繼續求職  沒有目標該怎麼辦',
 '兩家公司面試該如何抉擇',
 '我想轉職，但沒有個方向，想請問各位的想法',
 '職場其實是不想錄用中年大叔',
 '請問準備國考的這件事要寫在自傳裡嗎?',
 '想轉換跑道，嘗試其他工作',
 '該傾聽自己出去闖天下，還是傾聽家人放棄夢想?',
 '想嘗試其他工作，轉換跑道',
 '不想做作業員，但離目標工作還好遠',
 '想轉職至建築設計/室內設計，想知道必備條件是那些?',
 '稽核轉資安的入門及後續規劃',
 '面試設計相關職缺，履歷和作品集要怎麼準備？如何提升面試機會?',
 '轉型成前端設計師會不會比較有"錢"途？',
 nan,
 '請問倘若有前置協商的情況會影響求職的機會嗎？',
 '想了解作業流程與職責歸屬問題，以及該如何修飾離職原因',
 '想轉換跑道，嘗試其他工作',
 '面試幾間公司的業助，幾乎都是無聲卡，下步不知該怎麼走',
 '想轉職科技業卻不知能從何開始',
 '31歲沒有工作方向，投履歷都無下文']

In [161]:
len(recommend_beta('2384cc6e-2389-4be1-bc0d-84d682ea2714',30))

31